### **Setting GEE environnement**
Due to compatibility issues it is recommended to use a dedicated environment for the geemap library.

See instructions on how to do this here: https://geemap.org/installation/

In [ ]:
# Install the libraries if they are not installed (you can remove these lines if you have already installed them)
# !pip install earthengine-api geemap

# Charging the libraries
import ee
import geemap

# Authenticate and initialize Earth Engine session
# ee.Authenticate()  # Only required the first time or when the access token expires
ee.Initialize()

### **Visualisation mode for one roi in one year_of_interest**

In [ ]:
# Define the ROI and load the dataset
roi = ee.FeatureCollection('users/caslumali/UE801_Stage/MDD') # change the name of the ROIi : 'Cotriguacu' / 'Guaviare' / 'MDD' / 'Paragominas'
chirpsPrecipitation  = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

In [ ]:
def calculate_annual_precipitation_sum(year, roi):
    """Calculate the annual cumulative precipitation for a specific climate year."""
    start_date = ee.Date.fromYMD(year, 8, 1)  # Start of the climate year
    end_date = ee.Date.fromYMD(year + 1, 7, 31)  # End of the climate year
    filtered_year = chirpsPrecipitation.filterDate(start_date, end_date).map(lambda image: image.clip(roi))
    annual_precipitation_sum = filtered_year.reduce(ee.Reducer.sum()).rename('Precipitation')
    return annual_precipitation_sum

def calculate_stats(annual_precipitation_sum, roi):
    """Calculate max, min, and average precipitation within the ROI."""
    stats = annual_precipitation_sum.reduceRegion(
        reducer=ee.Reducer.minMax().combine(reducer2=ee.Reducer.mean(), sharedInputs=True),
        geometry=roi.geometry(),
        scale=5566,  # Match the resolution of CHIRPS data
        maxPixels=1e9
    )
    return stats.getInfo()

# Year of interest for the precipitation analysis
year_of_interest = 2000 # change the date for stats and visualisation

# Calculate the annual cumulative precipitation
annual_precipitation_sum = calculate_annual_precipitation_sum(year_of_interest, roi)

# Calculate and print precipitation statistics
stats = calculate_stats(annual_precipitation_sum, roi)
print(f"Precipitation values for {year_of_interest} - {year_of_interest + 1}\n", stats)

In [ ]:
# Calculate CHIRPS statistics for the year of interest to generate dynamic visualization
stats = calculate_stats(annual_precipitation_sum, roi)

# Define visualization parameters based on the calculated statistics
precipitation_vis = {
    'min': stats['Precipitation_min'],
    'max': stats['Precipitation_max'],
    'palette': ['blue', 'limegreen', 'yellow', 'orange', 'red']
}

# View the NDVI image on the map
Map = geemap.Map()
Map.addLayer(annual_precipitation_sum, precipitation_vis, 'Annual Precipitation Sum')
Map.centerObject(roi, 7) # Change the zoom depending on the area
Map

### **Looping for export NDVI for each ROI**

In [ ]:
# Define ROIs in a list
# Export ROIs have a 10km buffer
rois = {
    'Cotriguacu': ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu_10km'),
    'Guaviare': ee.FeatureCollection('users/caslumali/UE801_Stage/Guaviare_10km'),
    'MDD': ee.FeatureCollection('users/caslumali/UE801_Stage/MDD_10km'),
    'Paragominas': ee.FeatureCollection('users/caslumali/UE801_Stage/Paragominas_10km'),
}

In [ ]:
from datetime import datetime

# Get current year
current_year = datetime.now().year

# Loop for each ROI
for roi_name, roi in rois.items():
    print(f'Processing {roi_name}...')
    for year in range(2000, current_year):  # Goes to the current year
        # If we are in the current year and processing the last year, skip the combination
        if year == current_year - 1:
            continue  # Skip to the next iteration of the loop
        
        # Calculate annual precipitation sum.
        annual_precipitation_sum = calculate_annual_precipitation_sum(year, roi).toDouble()
        
        # Set up export parameters.
        description = f'{roi_name}_CHIRPS_Precipitation_Annual_Sum_{year}_{year + 1}'
        file_name_prefix = f'{roi_name}_CHIRPS_Precipitation_Annual_Sum_{year}_{year + 1}'
        
        # Start export task.
        task = ee.batch.Export.image.toDrive(
            image=annual_precipitation_sum,
            description=description,
            fileNamePrefix=file_name_prefix,
            folder=f'CHIRPS_Precipitation_{roi_name}',
            scale=5566,
            region=roi.geometry(),
            crs='EPSG:4326',
            fileFormat='GeoTIFF',
            maxPixels=1e9
        )
        task.start()
        print(f'Export task started for {roi_name} in the year: {year}')